In [65]:
import pandas as pd
import numpy as np

#### Information on data:
- Booking data for 2 hotels 
- Both hotels are located in Portugal: H1 at the resort region of Algarve and H2 at the city of Lisbon
- Source: https://www.sciencedirect.com/science/article/pii/S2352340918315191

### Read clean data from csv

In [3]:
data = pd.read_csv('../data/Hotels.csv')

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
data.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Hotel,Arrival_Date_MonthN,ArrivalDateFull,TotalNights
0,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,Not applicable,Not applicable,0,Transient,75.0,0,0,Check-Out,2015-07-02,H1,7,2015-07-01,1
1,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,Not applicable,0,Transient,75.0,0,0,Check-Out,2015-07-02,H1,7,2015-07-01,1
2,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,Not applicable,0,Transient,98.0,0,1,Check-Out,2015-07-03,H1,7,2015-07-01,2
3,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,Not applicable,0,Transient,98.0,0,1,Check-Out,2015-07-03,H1,7,2015-07-01,2
4,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,Not applicable,Not applicable,0,Transient,107.0,0,0,Check-Out,2015-07-03,H1,7,2015-07-01,2


### Feature engineering

In [100]:
data_drop = data.copy()

Drop irrelevant columns

In [101]:
data_drop.drop(['TotalNights', 'ArrivalDateFull', 'Arrival_Date_MonthN', 'ReservationStatusDate', 'TotalOfSpecialRequests', 'AssignedRoomType'], axis=1, inplace=True)

In [102]:
print(data_drop.shape)
data_drop.head()

(118561, 29)


,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,ReservationStatus,Hotel
0,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,0,No Deposit,Not applicable,Not applicable,0,Transient,75.0,0,Check-Out,H1
1,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,0,No Deposit,304,Not applicable,0,Transient,75.0,0,Check-Out,H1
2,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,0,No Deposit,240,Not applicable,0,Transient,98.0,0,Check-Out,H1
3,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,0,No Deposit,240,Not applicable,0,Transient,98.0,0,Check-Out,H1
4,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,0,No Deposit,Not applicable,Not applicable,0,Transient,107.0,0,Check-Out,H1


Reduce Country, Company and Agent columns

In [103]:
data_drop.Agent = [agent.strip() for agent in data_drop.Agent]

In [104]:
top = list(data_drop.Agent.value_counts().sort_values().tail(11).index)

In [105]:
top

['8', '28', '241', '250', '6', '7', '14', '1', '240', 'Not applicable', '9']

In [96]:
for agent in top:
    if agent not in top:
        data_drop.Agent = data_drop.Agent.str.replace(agent, 'Not applicable')

### Building classification model (unsupervised ML)

### Quick simulation for Linear regression

In [19]:
df=data.copy()

In [20]:
y = df["IsCanceled"]

In [21]:
df['IsCanceled'].dtype == 'int64'

num = []
cat = []
for col in df.columns:
    if df[col].dtype in ["int64", "float64"]:
        num.append(col)
    else:
        cat.append(col)

In [22]:
cat = ['ArrivalDateMonth',
 'Meal',
 'MarketSegment',
 'DistributionChannel',
 'ReservedRoomType',
 'AssignedRoomType',
 'DepositType',
 'CustomerType']

In [24]:
n = 12
print(cat[n])
df[cat[n]]

IndexError: list index out of range

In [26]:
num = ['LeadTime',
 'ArrivalDateYear',
 'ArrivalDateWeekNumber',
 'ArrivalDateDayOfMonth',
 'StaysInWeekendNights',
 'StaysInWeekNights',
 'Adults',
 'Children',
 'Babies',
 'IsRepeatedGuest',
 'PreviousCancellations',
 'PreviousBookingsNotCanceled',
 'BookingChanges',
 'DaysInWaitingList',
 'ADR',
 'RequiredCarParkingSpaces',
 'TotalOfSpecialRequests']

In [27]:
ml = [df[col] for col in num] + [pd.get_dummies(df[col]) for col in cat]
df_ml = pd.concat(ml, axis=1)

In [28]:
len(df_ml.columns)

76

In [29]:
n = 35000

y_train = df["IsCanceled"].iloc[0:n]
X_train = df_ml.iloc[0:n] 

y_test = df["IsCanceled"].iloc[n:]
X_test = df_ml.iloc[n:] 

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

df_ml.columns

lr = LogisticRegression()
rf = RandomForestClassifier()
#lr.fit(X_train,y_train)
rf.fit(X_train,y_train)

/Users/laumaustupa/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
from sklearn.metrics import confusion_matrix


In [32]:
pred = rf.predict(X_test)

confusion_matrix(y_test, pred)

array([[34337, 16947],
       [ 8282, 24820]])

In [33]:
df['IsCanceled'].value_counts(normalize=True)

0    0.62958
1    0.37042
Name: IsCanceled, dtype: float64